In [380]:
print(f"Logstash-Filebeat from the B&D Team to the ES Team")

Logstash-Filebeat from the B&D Team to the ES Team


In [381]:
#!pip install elasticsearch==7.13.0

In [382]:
from elasticsearch import Elasticsearch
import os
import json
import pandas as pd
import jsondiff
import logging
from dotenv import load_dotenv
import socket
import requests
import warnings
warnings.filterwarnings("ignore")

In [383]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

In [384]:
def get_headers():
    ''' Elasticsearch Header '''
    return {
            'Content-type': 'application/json', 
            'Authorization' : '{}'.format(os.getenv('BASIC_AUTH')),
            # 'Connection': 'close'
    }

In [385]:
source_dict = {
    # 'qa' : os.getenv('qa_env'),
    'qa' : 'localhost|localhost:9200',
    'prod' : os.getenv('prod_env'),
}

In [386]:
env = 'qa'
# env = 'prod'

In [387]:
def get_env_es_list():
    host_lists_value = source_dict.get(env).split(",")
    env_host_url = []
    env_host_name = {}
    for element in host_lists_value:
        env_name = element.split('|')[0]
        env_host_url_str = element.split('|')[1]
        env_host_url.append(env_host_url_str)
        env_host_name.update({env_host_url_str : env_name})
    return env_host_url, env_host_name

In [388]:
env_host_url, env_host_name = get_env_es_list()

In [389]:
# print(env_host_url)
print(env_host_name)

{'localhost:9200': 'localhost'}


In [390]:
df_dict = {
    'env' : env_host_name.values(),
    'hostname' : env_host_name.keys()
}

In [391]:
df = pd.DataFrame.from_dict(df_dict)
# df.head(10)
# display(df)

In [ ]:
def get_es_instance(host):
    es_client = Elasticsearch(hosts="{}".format(host), headers=get_headers(), timeout=5,  verify_certs=False)
    return es_client


try:
    df_indices_env_dict = {}
    lookup_env, lookup_result = [], []
    
    ''' Source cluster '''
    for es_host, env_name in env_host_name.items():
        es_obj_s_client = get_es_instance(es_host)
        # source_idx_lists = es_obj_s_client.indices.get("*")
        source_idx_lists = es_obj_s_client.cat.indices(index='logstash*', h='index', s='index:desc').split()
        # print(source_idx_lists)
        if len(source_idx_lists) > 0:
            lookup_env.append(env_name)
            lookup_result.append('Exist')
        # break
except Exception as e:
    print(e)

In [ ]:
df_indices_env_dict.update({
            'env' : lookup_env,
            'result' : lookup_result
})

In [ ]:
df = pd.DataFrame.from_dict(df_indices_env_dict)
# df.head(10)
display(df)